To do
* Organize Webscraping code
* Revise style/adjective and artist list
* Create indicator variables

[Reference used for webscraping Reddit](https://towardsdatascience.com/scraping-reddit-data-1c0af3040768)

[PRAW Documentation](https://praw.readthedocs.io/en/latest/index.html)

Installing libraries

In [ ]:
x = pd.Series(["comic book"])
for a in x:
  a = a.upper()
  print(a)
x.str.contains('comic book')

COMIC BOOK


0    True
dtype: bool

In [ ]:
pip install praw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 188 kB 5.2 MB/s 
     |████████████████████████████████| 55 kB 3.1 MB/s 


Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import praw

In [ ]:
cred = np.genfromtxt('cred.txt',dtype=str,delimiter=',')

Creating a Reddit Instance (need to hide the details in the cell later at some point)

In [ ]:
reddit = praw.Reddit(client_id=cred[0], client_secret=cred[1], user_agent=cred[2])

Let's play around with this

In [ ]:
hot_posts = reddit.subreddit('StableDiffusion').hot(limit=100)
for post in hot_posts:
  print(post.id)

In [ ]:
submission = reddit.submission(id='xcq819')
for y in submission.comments:
  print(y.body)

In [ ]:
type(submission.comments[0].body)

Let's try scraping the data into a dataframe by a certain flair.
Hmmm it seems that users are putting the prompts in the following places:


*   title
*   in a body of text where the images are linked elsewhere
*   comments

We're going to need to account for all of this



In [ ]:
posts = []
sd_subreddit = reddit.subreddit('StableDiffusion')
for post in sd_subreddit.hot(limit=1000):
  if post.link_flair_text == 'Prompt Included':
    posts.append([post.title,post.comments,post.url,post.id,post.score,post.num_comments,post.over_18])
    
posts = pd.DataFrame(posts,columns=['title',
                                    'comments',
                                    'url',
                                    'id',
                                    'score',
                                    'num_comments',
                                    'nsfw'])

In [ ]:
posts.head()

,title,comments,url,id,score,num_comments,nsfw
0,'This looks like a nice place for a nap.',"(iqzlmuh, ir0l1pa, ir0sc0q)",https://i.redd.it/if84ow9nnqr91.png,xv86kw,85,7,False
1,Just some cute hand sewn pumpkins,"(ir0x84g, ir190zu)",https://www.reddit.com/gallery/xvhq8v,xvhq8v,11,2,False
2,How to put HEAVY BlingBling golden chains arou...,"(iqz85k2, iqzhgiq, iqzs27l, iqzwn3g)",https://www.reddit.com/gallery/xv5bbd,xv5bbd,63,10,False
3,Dragonball Sesamestreet mash-up,(ir0euko),https://i.redd.it/v1hpyux5rqr91.jpg,xv8hcz,33,2,False
4,Christoph Waltz as a high ranking Imperial off...,(),https://i.redd.it/iha29x8vgtr91.jpg,xvjrpd,6,0,False


In [ ]:
posts.shape

(162, 7)

In [ ]:
comments = []
for i in posts.id:
  submission = reddit.submission(id=i)
  for j in submission.comments:
    comments.append([j.body,i])
comments = pd.DataFrame(comments,columns=['text','id'])  


In [ ]:
comments.iloc[0,0]

'Created with Stable Diffusion using the NMKD Stable Diffusion GUI - AI Image Generator.\r  \n\r  \nPrompt:\r  \nfemale ent, insanely detailed and intricate, hypermaximalist, elegant, ornate, hyper realistic, super detailed,, 8K, vivid colors, sunrise, open sky, black bear\r  \n\\-1029314660 -scale 7.50 -k euler a  \n\n\nYes, the female ent is supposed to be in there somewhere, but I never found her either.'

In [ ]:
comments.to_csv('/content/comments.csv')

It looks like comments are being cut off once converted to a csv

In [ ]:
comments.to_excel('/content/comments.xlsx')

Okay it looks like the whole comment is intact if you leave it in an Excel file format.

Okay so it looks like to get past the 1000 limit we can use PSAW (Pushshift) and get an index of id and then use PRAW to look at those posts and comments more in-depth. So let's figure out how the API works:

In [ ]:
pip install psaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from psaw import PushshiftAPI

In [ ]:
api = PushshiftAPI(reddit)

In [ ]:
test = list(api.search_submissions(q='prompt',subreddit='StableDiffusion',limit = 10))
test

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



[Submission(id='xvo7wy'),
 Submission(id='xvnsf1'),
 Submission(id='xvmgsv'),
 Submission(id='xvjm84'),
 Submission(id='xvjhd2'),
 Submission(id='xvjckt'),
 Submission(id='xvj8m2'),
 Submission(id='xvgm0a'),
 Submission(id='xvf1bi'),
 Submission(id='xvefa6')]

In [ ]:
#Getting the id from the first element in the list
test[0].id

'xvo7wy'

In [ ]:
len(test)

10

In [ ]:
comments = []
for a in test:
  submission = reddit.submission(id = a.id)
  for b in submission.comments:
    comments.append([b.body,a.id])
comments = pd.DataFrame(comments,columns=['text','id'])

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
comments.shape

(6, 2)

In [ ]:
comments.to_excel('/content/test_comments.xlsx')

This is all the new working stuff

In [ ]:
#Grab a list of ids of posts from the Stable Diffusion subreddit
submissionList = list(api.search_submissions(subreddit='StableDiffusion'))

# Create temporary list
possiblePromptPost = []

# Get posts with "Prompt Included" flair
for post in submissionList:
  if post.link_flair_text == 'Prompt Included':
    possiblePromptPost.append(post)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
temp = []
for submission in possiblePromptPost:
  post = reddit.submission(id = submission)
  temp.append([post.title,post.id,post.selftext,post.comments])
#reddit.submission(id='xvf1bi').title
df = pd.DataFrame(temp, columns = ['title','id','text','comments'])
df

Streaming output truncated to the last 5000 lines.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.r

,title,id,text,comments
0,Man needlessly standing on roof overlooking cy...,xvrn98,,(ir2nbtb)
1,"first attempt at art nouveau prompts, in comments",xvrbxa,,(ir2l71a)
2,Lovecraftian Behemoth,xvqu7g,,(ir2imtj)
3,Henry Cavill medieval times(?,xvqpvg,,(ir2hh2s)
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,,(ir2agth)
...,...,...,...,...
2385,Renaissance art of Keanu Reeves,woy7hy,,"(ikdgwmu, ikdl3yb, ikhe3c9)"
2386,Jazz robots.,wo3wm3,,"(ik8r5th, ik8qvj5, ikal2cb, ikcoao7, ikhgxtj)"
2387,Raves celebrating the day of the dead,wnei33,,"(ik4jkze, ik5eqzu)"
2388,Joker artwork,wm97qi,,"(ijxy1k4, ijzqeys)"


In [ ]:
df.to_excel('/content/prompt_data.xlsx')

In [ ]:
df = pd.read_excel('/content/prompt_data.xlsx')

In [ ]:
df.head()

,Unnamed: 0,title,id,text,comments
0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,<praw.models.comment_forest.CommentForest obje...
1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,<praw.models.comment_forest.CommentForest obje...
2,2,Lovecraftian Behemoth,xvqu7g,NaN,<praw.models.comment_forest.CommentForest obje...
3,3,Henry Cavill medieval times(?,xvqpvg,NaN,<praw.models.comment_forest.CommentForest obje...
4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,<praw.models.comment_forest.CommentForest obje...


In [ ]:
print(df.comments[0])

In [ ]:
# Getting one comment
submission = reddit.submission(id = df.id[0])
for comment in submission.comments:
  print(comment.body)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Generated with:

> Portrait of a man overlooking vast cyberpunk city at night, neon signs, vivid colors, sci-fi, cinematic lighting, cyberpunk 2077, illustration, art by Rebecca Guay and Yoshitaka Amano and Syd Mead
Steps: 30, Sampler: Euler a, CFG scale: 10

Then fed through img2img and SD Upscaler with 2 to 3 denoising strength with DPM2 a sampler at 100 (150?) steps with the same parameter.

AUTOMATIC1111's web-ui


In [ ]:
#Getting the first comments of the 50 posts if any

#Create temporary list
temp = []

#Iterate through the first 50 posts
for id in df.id:

  #Create a submission object with the id
  submission = reddit.submission(id = id)

  #Check if the comment tree is empty
  if (len(submission.comments) == 0):
    temp.append("NA")
  else:
    # Create a temp list to store OP's comments' epoch times
    utc = []

    # Create a temp list to store OP's comments' ids
    op_comment_ids = []

    # Iterate through all the comments
    for comment in submission.comments:

      # Check if the comment is the OP
      if comment.is_submitter:

        # Add the epoch time to the list
        utc.append(comment.created_utc)

        # Add the comment's id to the op comment id list
        op_comment_ids.append(comment.id)

    for op_comment_id in op_comment_ids:
      curr_comment = reddit.comment(op_comment_id)
      # Check if the comment
      if curr_comment.created_utc == min(utc):

        # Append the comment to the temp list
        temp.append(curr_comment.body)
    
    # Append "NA" if there are no OP comments 
    if len(utc) == 0:
      temp.append("NA")
  
df.loc[:,'comments'] = temp
df.to_excel('/content/new_prompt_data.xlsx')


Streaming output truncated to the last 5000 lines.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.r

In [ ]:
temp

In [ ]:
df.loc[:,'comments']

0       <praw.models.comment_forest.CommentForest obje...
1       <praw.models.comment_forest.CommentForest obje...
2       <praw.models.comment_forest.CommentForest obje...
3       <praw.models.comment_forest.CommentForest obje...
4       <praw.models.comment_forest.CommentForest obje...
                              ...                        
2385    <praw.models.comment_forest.CommentForest obje...
2386    <praw.models.comment_forest.CommentForest obje...
2387    <praw.models.comment_forest.CommentForest obje...
2388    <praw.models.comment_forest.CommentForest obje...
2389    <praw.models.comment_forest.CommentForest obje...
Name: comments, Length: 2390, dtype: object

In [ ]:
len(temp)

50

In [ ]:
reddit.comment(reddit.submission(id = 'xw3n50').comments[0].id).body

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



"*No downvotes, please. It's a waste of good suffering.*"

The problem we have here is that the first comment might not always be the where the prompt is as it takes the comments in sorted by best. We want to find a way to get the OP's top level comment which is usually where the comment is

In [ ]:
submission = reddit.submission(id = "xw3n50")

utc = 9999999999999
for x in submission.comments:
  
  #print(x.created_utc)
  if x.created_utc < utc:
    utc = x.created_utc
  
  # Check if the comment is OP's
  #print(x.is_submitter)
  #if x.is_submitter:
    #print(x.body)
#print(utc)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



1665004881.0


In [ ]:
df[df.id == 'xvo91q']

,Unnamed: 0,title,id,text,comments
6,6,D&D character animation with Stable Diffusion,xvo91q,NaN,<praw.models.comment_forest.CommentForest obje...


In [ ]:
temp

["Generated with:\n\n> Portrait of a man overlooking vast cyberpunk city at night, neon signs, vivid colors, sci-fi, cinematic lighting, cyberpunk 2077, illustration, art by Rebecca Guay and Yoshitaka Amano and Syd Mead\nSteps: 30, Sampler: Euler a, CFG scale: 10\n\nThen fed through img2img and SD Upscaler with 2 to 3 denoising strength with DPM2 a sampler at 100 (150?) steps with the same parameter.\n\nAUTOMATIC1111's web-ui",
 "Prompts:\nOil painting of 'self daydreaming whilst watching the clouds', Alphonse Mucha, intricate gold leaf border, tarot card, natural lighting, rule of thirds, 4k UHD",
 "Ultra realistic Illustration of a Lovecraftian behemoth of translucent slime envelops a 1850s steampunk hamlet, psychedelic pastels, eldritch horror, sharp focus, soft focus, asymmetrical, green and purple ethereal lighting, Ayami Kojima, Bernard Buffet, Hans Bellmer\n\nSteps: 20, Sampler: Euler a, CFG scale: 16\n\nthen img2img lose the artists and illustrations and swap in\n\nPhotorealist

In [ ]:
df.head(10).id
pd.notna(reddit.submission(id = df.id[0]).comments)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



True

In [ ]:
df = pd.read_excel('/content/new_prompt_data.xlsx')

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,title,id,text,comments
0,0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,2,2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,3,3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."


In [ ]:
example = 'Prompt: a man :'
example.find(':')

6

In [ ]:
type(df.comments[1])

float

In [ ]:
for comment in df.head(100).comments:
  if isinstance(comment,str):
    if (comment.find('Prompt') > -1):
      print(comment[(comment.find('Prompt') + 6):500])

: android pippi longstocking samurai, neo-tokyo, futuristic, 80mm, depth of field, cinematic, synthwave, depth of field

Width: 768  
Height: 512  
Seed: 9044657  
Steps: 80  
Guidance Scale: 12.0  
Prompt Strength: 0.8  
Use Face Correction: GFPGANv1.3  
Sampler: plms

I've edited this one in Photoshop since the left side of the left side of the face was a big janky. You'll find the original and others I made with a [similar prompt here](https://drive.google.com/drive/folders/1FIuafSoirpq
: “A cute hand sewn pumpkin”

Added things like: wool, fabric, velvet, fuzzy, knit, woven, plaid, Frankenstein, country, autumn, amish, by Matisse, by Ralph Lauren, etc…
 = a full body potroit of one beautiful female pirate looking at a huge pirate city in the distance, insane level of detail, cinematic, composition, sense of awe and scale, artstation HQ, 16K
: ornate stone cube drenched in black goop and machinery, {red light|blue light}, lovecraftian, horror art, 4K, dark art, dramatic lighting

Wa

References:

[Submission attributes](https://praw.readthedocs.io/en/latest/code_overview/models/submission.html)

[Getting posts with a specific flair](https://www.reddit.com/r/redditdev/comments/x4vydg/how_do_i_pull_posts_with_a_specific_flair_from_a/)

[Access more than 100 posts](https://www.reddit.com/r/redditdev/comments/t3j440/how_to_access_more_than_100_posts_with_praw/)

[How to Scrape Large Amounts of Reddit Data](https://medium.com/swlh/how-to-scrape-large-amounts-of-reddit-data-using-pushshift-1d33bde9286)

[Getting many/all submissions from a subreddit using PRAW/PSAW/pushshift](https://www.reddit.com/r/redditdev/comments/knbns9/getting_manyall_submissions_from_a_subreddit/)

[Using Pushshift API for data analysis on Reddit](https://medium.com/mcd-unison/using-pushshift-api-for-data-analysis-on-reddit-b08d339c48b8)

[Video: Reddit Scraping using PRAW and Pushshift](https://www.youtube.com/watch?v=a3zQdlLwEb4)

[Pushshift Reddit API v4.0 Documentation](https://reddit-api.readthedocs.io/en/latest/#search-parameters-for-comments)

[New to Pushshift? Read this! FAQ](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/)


